In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import glob
import cartopy.crs as ccrs
import matplotlib as mpl
from scipy.interpolate import griddata
import numpy as np
import time
import copy as cp
import matplotlib.dates as mdates
from datetime import datetime
import cmocean
import cartopy.feature as cfeature
import matplotlib.colors as mcolors
from matplotlib.ticker import FormatStrFormatter
import csv
import cftime
import matplotlib.ticker as mticker
from datetime import datetime as dtime
import datetime
import pandas as pd
from xorca.lib import load_xorca_dataset
import copy as cp
from xgcm import Grid
import xgcm


In [2]:
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

In [3]:
def re_grid_ORCA_Armitage(Aplot, lon, lat):
    # Make the new grid
    lons_1d=lon
    lats_1d=lat
    lons_2d, lats_2d = np.meshgrid(lons_1d, lats_1d)
    lons = cp.deepcopy(Aplot.lons.values)
    #lons[lons<0] = lons[lons<0]+360
    lats = Aplot.lats.values
    temp=Aplot.values
    lons=lons.flatten()
    lats=lats.flatten()
    temp=temp.flatten()

    
    temp_new = griddata((lons, lats), temp, (lons_2d, lats_2d), method='linear')
    
    dint = xr.Dataset(data_vars = {'Lats_reg':lats_1d, 'Lons_reg_360':lons_1d})
    dint['val'] = (('Lats_reg', 'Lons_reg_360'), temp_new)
    return dint

# Datasets

## A-SSH

In [4]:
# early armitage
dArmitage1 = xr.open_dataset('/mnt/shackleton/storage3/nplanat/Data_SSH/CPOM_DOT.nc')
IntDate  = [str(dArmitage1.date.values[i]) for i in range(len(dArmitage1.date.values))]
time_c = [datetime.datetime.strptime(str(IntDate[i]),'%Y%m') for i in range(len(IntDate))]
time = [time_c[i] + datetime.timedelta(days = 15) for i in range(len(IntDate))]
dArmitage1['time'] = (('date'), time)
dArmitage1 = dArmitage1.drop('date').set_coords({'time'}).rename({'date':'time'})

dArmitage2 = dArmitage1.rename({'DOT':'DOT_smoothed'}).drop({'U', 'V'})
dArmitage2['DOT_smoothed'] = dArmitage2.DOT_smoothed-dArmitage2.DOT_smoothed.isel(time =  slice(96, 132)).mean() # 2011-01-01', '2013-12-31'

# late armitage
dArmitage = xr.open_dataset('/mnt/shackleton/storage3/nplanat/Data_SSH/Full_DOT_data_Arco.nc')
dArmitage = dArmitage.set_coords('lats')
dArmitage = dArmitage.set_coords('lons')
Lons_180 = dArmitage.lons.values
for x in range(334):
    for y in range(334):
        if Lons_180[x,y]>180:
            Lons_180[x,y] -=360
dArmitage['Lons_180'] = (('x', 'y'), Lons_180)
dArmitage = dArmitage.set_coords('Lons_180')
time_counter = cftime.num2date(dArmitage.time.values, dArmitage.time.attrs['Time_dimension'])
time_c = [datetime.datetime.strptime(str(time_counter[i]),'%Y-%m-%d %H:%M:%S') for i in range(len(time_counter))]
dArmitage['time_counter'] = (('time'), time_c)
dArmitage = dArmitage.swap_dims({'time':'time_counter'}).drop({'time'}).rename({'time_counter':'time'})

dArmitage4 = xr.Dataset(coords = {'time':dArmitage.time.values, 'lon':dArmitage2.lon.values, 'lat':dArmitage2.lat.values})
DOT = np.zeros((dArmitage4.time.shape[0], dArmitage4.lon.shape[0], dArmitage4.lat.shape[0]))
for t in range(dArmitage.time.shape[0]):
    DOT[t,:,:] = np.transpose(re_grid_ORCA_Armitage(dArmitage.DOT_smoothed.isel(time = t)-dArmitage.DOT_smoothed.sel(time =  slice('2011-01-01', '2013-12-31')).mean(), lon = dArmitage4.lon.values, lat = dArmitage4.lat.values).val.values)
dArmitage4['DOT_smoothed']  = (('time', 'lon', 'lat'), DOT)
dArmitage4['DOT_smoothed'] = (('time', 'lon', 'lat'), dArmitage4.DOT_smoothed.where(dArmitage4.lon<179.5, np.nan).values)

/tmp/ipykernel_383593/797491678.py:8: UserWarning: rename 'date' to 'time' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  dArmitage1 = dArmitage1.drop('date').set_coords({'time'}).rename({'date':'time'})


In [12]:
# together
dArmi_tot = xr.concat((dArmitage2.isel(time = slice(12, 8*12)), dArmitage4.drop_indexes({'time'})), dim = 'time') # '2004-01-01', '2010-12-31'
dArmi_tot['DOT_smoothed'] = (('lat', 'lon', 'time'), dArmi_tot.DOT_smoothed.where(dArmi_tot.lon<179.5, np.nan).values)

In [13]:
months = dArmi_tot.time.dt.month
dsshA = xr.Dataset(coords = {'lon':dArmi_tot.lon, 'lat':dArmi_tot.lat,  'month':months})
dsm = dArmi_tot.where(dArmi_tot.time.dt.year>=2005).where(dArmi_tot.time.dt.year<=2017).groupby('time.month').mean('time')
dsshA['ssh_A5B7_m'] = dsm.DOT_smoothed - dsm.DOT_smoothed.mean()

In [ ]:
#dsshA.to_netcdf('/storage/nplanat/DataSSH/Final_SSH_Armi_tot.nc')

## O-SSH

In [14]:
# opa 0
opa = 'opa0'
#L_4 = ['/mnt/shackleton/storage3/nplanat/Data_Glorys4_masks/mesh_hgr.nc', '/mnt/shackleton/storage3/nplanat/Data_Glorys4_masks/mask.nc','/storage/nplanat/Glorys4_masks/mesh_zgr.nc']
L_4 = ['/storage/nplanat/Data_G4_masks/mesh_mask_cropped2.nc']
dsOpa0 = load_xorca_dataset(data_files=sorted(glob.glob('/mnt/shackleton/storage3/nplanat/Data_SSH/ORAS/oras5/ORCA025/sossheig/'+opa+'/SSH_*')),
                            aux_files=L_4, update_orca_variables = {'sossheig' : {"dims":["t", "y_c", "x_c",]}})
dsOpa0['at'] = dsOpa0['e1t']*dsOpa0['e2t']
dsOpa0['au'] = dsOpa0['e1u']*dsOpa0['e2u']
dsOpa0['av'] = dsOpa0['e1v']*dsOpa0['e2v']
dsOpa0['af'] = dsOpa0['e1f']*dsOpa0['e2f']
dsOpa0 = dsOpa0.set_coords(('at', 'au', 'av', 'af'))

metrics = {
    ('X',): ['e1u', 'e1v'], # X distances
    ('Y',): ['e2v', 'e2u'], # Y distances
    ('X', 'Y'): ['at', 'au', 'av', 'af'] # Areas
}
coords={"X": {"center": "x_c", "right": "x_r"}, "Y":{'center':'y_c', 'right':'y_r'}}

gridOpa0 = xgcm.Grid(dsOpa0, periodic=["X"], coords = coords, metrics=metrics, boundary = 'fill', fill_value=np.nan)

opa = 'opa1'
dsOpa1 = load_xorca_dataset(data_files=sorted(glob.glob('/mnt/shackleton/storage3/nplanat/Data_SSH/ORAS/oras5/ORCA025/sossheig/'+opa+'/SSH_*')),
                            aux_files=L_4, update_orca_variables = {'sossheig' : {"dims":["t", "y_c", "x_c",]}})
dsOpa1['at'] = dsOpa1['e1t']*dsOpa1['e2t']
dsOpa1['au'] = dsOpa1['e1u']*dsOpa1['e2u']
dsOpa1['av'] = dsOpa1['e1v']*dsOpa1['e2v']
dsOpa1['af'] = dsOpa1['e1f']*dsOpa1['e2f']
dsOpa1 = dsOpa1.set_coords(('at', 'au', 'av', 'af'))

metrics = {
    ('X',): ['e1u', 'e1v'], # X distances
    ('Y',): ['e2v', 'e2u'], # Y distances
    ('X', 'Y'): ['at', 'au', 'av', 'af'] # Areas
}
coords={"X": {"center": "x_c", "right": "x_r"}, "Y":{'center':'y_c', 'right':'y_r'}}

gridOpa1 = xgcm.Grid(dsOpa1, periodic=["X"], coords = coords, metrics=metrics, boundary = 'fill', fill_value=np.nan)

opa = 'opa2'
dsOpa2 = load_xorca_dataset(data_files=sorted(glob.glob('/mnt/shackleton/storage3/nplanat/Data_SSH/ORAS/oras5/ORCA025/sossheig/'+opa+'/SSH_*')),
                            aux_files=L_4, update_orca_variables = {'sossheig' : {"dims":["t", "y_c", "x_c",]}})
dsOpa2['at'] = dsOpa2['e1t']*dsOpa2['e2t']
dsOpa2['au'] = dsOpa2['e1u']*dsOpa2['e2u']
dsOpa2['av'] = dsOpa2['e1v']*dsOpa2['e2v']
dsOpa2['af'] = dsOpa2['e1f']*dsOpa2['e2f']
dsOpa2 = dsOpa2.set_coords(('at', 'au', 'av', 'af'))

metrics = {
    ('X',): ['e1u', 'e1v'], # X distances
    ('Y',): ['e2v', 'e2u'], # Y distances
    ('X', 'Y'): ['at', 'au', 'av', 'af'] # Areas
}
coords={"X": {"center": "x_c", "right": "x_r"}, "Y":{'center':'y_c', 'right':'y_r'}}

gridOpa2 = xgcm.Grid(dsOpa2, periodic=["X"], coords = coords, metrics=metrics, boundary = 'fill', fill_value=np.nan)

opa = 'opa3'
dsOpa3 = load_xorca_dataset(data_files=sorted(glob.glob('/mnt/shackleton/storage3/nplanat/Data_SSH/ORAS/oras5/ORCA025/sossheig/'+opa+'/SSH_*')),
                            aux_files=L_4, update_orca_variables = {'sossheig' : {"dims":["t", "y_c", "x_c",]}})
dsOpa3['at'] = dsOpa3['e1t']*dsOpa3['e2t']
dsOpa3['au'] = dsOpa3['e1u']*dsOpa3['e2u']
dsOpa3['av'] = dsOpa3['e1v']*dsOpa3['e2v']
dsOpa3['af'] = dsOpa3['e1f']*dsOpa3['e2f']
dsOpa3 = dsOpa3.set_coords(('at', 'au', 'av', 'af'))

metrics = {
    ('X',): ['e1u', 'e1v'], # X distances
    ('Y',): ['e2v', 'e2u'], # Y distances
    ('X', 'Y'): ['at', 'au', 'av', 'af'] # Areas
}
coords={"X": {"center": "x_c", "right": "x_r"}, "Y":{'center':'y_c', 'right':'y_r'}}

gridOpa3 = xgcm.Grid(dsOpa3, periodic=["X"], coords = coords, metrics=metrics, boundary = 'fill', fill_value=np.nan)

opa = 'opa4'
dsOpa4 = load_xorca_dataset(data_files=sorted(glob.glob('/mnt/shackleton/storage3/nplanat/Data_SSH/ORAS/oras5/ORCA025/sossheig/'+opa+'/SSH_*')),
                            aux_files=L_4, update_orca_variables = {'sossheig' : {"dims":["t", "y_c", "x_c",]}})
dsOpa4['at'] = dsOpa4['e1t']*dsOpa4['e2t']
dsOpa4['au'] = dsOpa4['e1u']*dsOpa4['e2u']
dsOpa4['av'] = dsOpa4['e1v']*dsOpa4['e2v']
dsOpa4['af'] = dsOpa4['e1f']*dsOpa4['e2f']
dsOpa4 = dsOpa4.set_coords(('at', 'au', 'av', 'af'))

metrics = {
    ('X',): ['e1u', 'e1v'], # X distances
    ('Y',): ['e2v', 'e2u'], # Y distances
    ('X', 'Y'): ['at', 'au', 'av', 'af'] # Areas
}
coords={"X": {"center": "x_c", "right": "x_r"}, "Y":{'center':'y_c', 'right':'y_r'}}

gridOpa4 = xgcm.Grid(dsOpa4, periodic=["X"], coords = coords, metrics=metrics, boundary = 'fill', fill_value=np.nan)

/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "z" starting at index 2. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWar

/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserW

/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserW

/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserW

/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserW

...

/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserW

/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserW

/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserW

/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserW

/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/aos/home/nplanat/miniconda3/envs/Analysis2/lib/python3.10/site-packages/xarray/core/dataset.py:270: UserW

In [15]:
dsOPA = xr.concat([dsOpa0,dsOpa1, dsOpa2, dsOpa3, dsOpa4], dim = 'int')
dsOPA_mean = dsOPA.mean(dim ='int')
dsOPA_mean_c = dsOPA_mean.where(dsOPA_mean.t.dt.year>=2005).where(dsOPA_mean.t.dt.year<=2017).groupby('t.month').mean('t')

In [17]:
months = dsOPA_mean.t.dt.month
dssh = xr.Dataset(coords = {'x_c':dsOPA_mean.x_c, 'y_c':dsOPA_mean.y_c,'month':months})
dsm = dsOPA_mean.where(dsOPA_mean.t.dt.year>=2005).where(dsOPA_mean.t.dt.year<=2017).groupby('t.month').mean('t')
dssh['ssh_A5B7_m'] = dsm.sossheig - dsm.sossheig.mean()
dsm = dsOPA_mean.where(dsOPA_mean.t.dt.year>=1975).where(dsOPA_mean.t.dt.year<=2017).groupby('t.month').mean('t')
dssh['ssh_all_m'] = dsm.sossheig - dsm.sossheig.mean()

In [ ]:
#dssh.to_netcdf('/storage/nplanat/DataSSH/Final_SSH_ORAS_2.nc')

## W-SSH

In [18]:
# get grid WOA to interpolate over it
dsT = xr.open_dataset('/storage/nplanat/DataWOA/woa18_A5B7_t00_01.nc', decode_times = False)
dsshm = xr.Dataset(coords = {'lat':dsT.lat.values, 'month':np.arange(1, 13), 'lon':dsT.lon.values})

In [19]:
for (eps, mid_lat) in [(3, 78.375)]:
    print(eps, mid_lat)
    SSH_MM = np.zeros((12, len(dsT.lon.values), len(dsT.lat.values)))
    for m in range(12):  

        ssh_O = dssh.ssh_A5B7_m.isel(month = m)
        ssh_A = dsshA.ssh_A5B7_m.isel(month = m)

        latA = ssh_O.llat_cc.values.reshape((ssh_O.llat_cc.shape[0]*ssh_O.llat_cc.shape[1], ))
        lonA = ssh_O.llon_cc.values.reshape((ssh_O.llon_cc.shape[0]*ssh_O.llon_cc.shape[1], ))
        values = ssh_O[:,:].values.reshape((ssh_O.llat_cc.shape[0]*ssh_O.llat_cc.shape[1], ))
        points = np.transpose(np.array((latA,lonA)))
        xx, yy = np.meshgrid(dsT.lat.values, dsT.lon.values)
        SSH_O = griddata(points, values, (xx, yy), method = 'linear')

        latA = ssh_A.lat.values
        lonA = ssh_A.lon.values
        LonA, LatA = np.meshgrid(lonA, latA)
        LonA = LonA.flatten()
        LatA = LatA.flatten()
        values = ssh_A[:,:].values.flatten()
        points = np.transpose(np.array((LatA,LonA)))
        xx, yy = np.meshgrid(dsT.lat.values, dsT.lon.values)
        SSH_A = griddata(points, values, (xx, yy), method = 'linear')
        ind_i = np.argmin(np.abs(dsT.lat.values - mid_lat+eps))
        ind_f = np.argmin(np.abs(dsT.lat.values - mid_lat-eps))
        y = dsT.lat[ind_i:ind_f].values
        coeff = 1/2*(1+np.cos(np.pi*((y-mid_lat)/(2*eps)-1/2)))
        y2 = dsT.lat[ind_f:].values
        y3 = dsT.lat[:ind_i].values
        co = np.concatenate((np.zeros(len(y3)), coeff, np.ones(len(y2))))
        Y = dsT.lat.values
        ones = np.ones(len(Y))
        SSH_MM[m,:,:] = np.nansum([SSH_O*co[np.newaxis, :], SSH_A*(ones - co)[np.newaxis, :]], axis = 0)
    dsshm['ssh_mixed_'+str(int(eps))+'_'+str(int(mid_lat))] = (('month', 'lon', 'lat'), SSH_MM)

3 78.375


In [ ]:
#dsshm.to_netcdf('/mnt/shackleton/storage3/nplanat/Data_SSH/Final2_SSH_mix.nc')